In [1]:
# use torch_2022

import numpy as np
#import pandas as pd
import pickle
import time
import os
import copy
import sys
#import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, models, transforms
from torchvision.io import read_image


# models - you do need the weights for the transformations.
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from torchvision.models import regnet_x_8gf, RegNet_X_8GF_Weights
from torchvision.models import swin_t, Swin_T_Weights
from torchvision.models import wide_resnet50_2, Wide_ResNet50_2_Weights

#import wandb

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)
print(sys.version)
print(torch.__version__)
print(torchvision.__version__)

cpu
3.8.13 (default, Mar 28 2022, 11:38:47) 
[GCC 7.5.0]
1.12.1
0.13.1


Make a loop: 

pseudo python code


For each model:
    For each fature:
        feature_list = []
        img_list = []
        for img:
            score = model(img)

            feature_list.add(score)
            img_list.add(img)

        df = pd.Dataframe({feature : feature_list, 'img_id' : img_list})
        df.to_pkl(.../model_feature.pkl)





In [2]:

# BUT HERE IT SHOULD JUST BE LOAD A PRETRAINED MODEL!!!!!!! # ---------------------------------------

#should be terminal inputs:

# choose one model
model_name = 'convnext_tiny' # last block is called "classifier" 
# model_name = 'efficientnet_v2_s' # last block is called "classifier" 
# model_name = 'regnet_x_8gf' # last block is called "fc" 
# model_name = 'swin_t'  # last block is called "head" 
# model_name = 'wide_resnet50_2'  # last block is called "fc" 

attribute = 'all_mass_protest' #

hyperparameters = {
"model_name" : model_name,
"attribute" : attribute,
"batch_size" : 32}


In [3]:
# NOW YOU DO NOT NEED TRAIN; TEST OR VAL AS EVERYTHING IS ALL IMAGES AND NO EXTRA TRANSFORAMTIONS:
# ALSO you do not have labels

class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):

        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels[idx, 0])
        image = read_image(img_path)
        label = None
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
def change_head(model_name, model, num_classes):

    if model_name == 'convnext_tiny':
        model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes, bias=False).to(device)
        print(f'new head: {model.classifier[2]}')

    elif model_name == 'efficientnet_v2_s':
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes, bias=False).to(device)
        print(f'new head: {model.classifier[1]}')

    elif model_name == 'regnet_x_8gf':
        model.fc = nn.Linear(model.fc.in_features, num_classes, bias=False).to(device)
        print(f'new head: {model.fc}')

    elif model_name == 'swin_t':
        model.head = nn.Linear(model.head.in_features, num_classes, bias=False).to(device)
        print(f'new head: {model.head}')

    elif model_name == 'wide_resnet50_2':
        model.fc = nn.Linear(model.fc.in_features, num_classes, bias=False).to(device)
        print(f'new head: {model.fc}')

    elif model_name == 'squeezenet1_1' :
        model.classifier[1] = nn.Conv2d(model.classifier[1].in_channels, num_classes, kernel_size=(1, 1), stride=(1, 1), bias=False).to(device)
        print(f'new head: {model.classifier[1]}')

    elif model_name == 'shufflenet_v2_x0_5' :
        model.fc = nn.Linear(model.fc.in_features, num_classes, bias=False).to(device)
        print(f'new head: {model.fc}')

    elif model_name == 'mnasnet0_5' :
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes, bias=False).to(device)
        print(f'new head: {model.classifier[1]}')    

    elif model_name == 'mobilenet_v3_small' :
        model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes, bias=False).to(device)
        print(f'new head: {model.classifier[3]}')

    else:
        print('Unddefined model name...')


In [5]:
model_dir = "/home/simon/Documents/Bodies/data/done_RA_models/"
model_name = f'{hyperparameters["model_name"]}_{hyperparameters["attribute"]}.pth'
model_path = f'{model_dir}{model_name}' 

print(model_path)

/home/simon/Documents/Bodies/data/done_RA_models/convnext_tiny_all_mass_protest.pth


In [6]:
# JUST LOAD PRETRAINED MODEL

def get_model(hyperparameters):

    weight_dict = {'convnext_tiny': ConvNeXt_Tiny_Weights.DEFAULT,
                'efficientnet_v2_s' : EfficientNet_V2_S_Weights.DEFAULT,
                'regnet_x_8gf' : RegNet_X_8GF_Weights.DEFAULT,
                'swin_t' : Swin_T_Weights.DEFAULT,
                'wide_resnet50_2' : Wide_ResNet50_2_Weights.DEFAULT}

    model_dict = {'convnext_tiny': convnext_tiny(weights = weight_dict['convnext_tiny']).to(device),
                'efficientnet_v2_s' : efficientnet_v2_s(weights = weight_dict['efficientnet_v2_s']).to(device),
                'regnet_x_8gf' : regnet_x_8gf(weights = weight_dict['regnet_x_8gf']).to(device),
                'swin_t' : swin_t(weights = weight_dict['swin_t']).to(device),
                'wide_resnet50_2' : wide_resnet50_2(weights = weight_dict['wide_resnet50_2']).to(device)}

    # model_name = hyperparameters['model_name']

    model_dir = "/home/simon/Documents/computerome/done_RA_models/"
    model_name = f'{hyperparameters["model_name"]}_{hyperparameters["attribute"]}'
    model_path = f'{model_dir}{model_name}'  # .pth

    #model = torch.load(model_path)
    #model.eval()

    # model_SD_dir = "/home/simon/Documents/Bodies/data/done_RA_models/"
    # model_SD_name = f"{hyperparameters['model_name']}_{hyperparameters['attribute']}"

    model = model_dict['convnext_tiny']
    change_head('convnext_tiny', model, 1)
    
    model.load_state_dict(torch.load(f'{model_dir}{model_name}_SD.pth', map_location=torch.device('cpu')))
    model.eval()

    return(model, weight_dict)

In [7]:
model, weight_dict = get_model(hyperparameters)

new head: Linear(in_features=768, out_features=1, bias=False)


In [ ]:
# data loader
def make_loader(batch_size, weights, attribute):

    data_transform = transforms.Compose([weights.transforms()])

    dict_dir = '/home/simon/Documents/Bodies/data/RA/dfs/' #local
    img_dir = '/media/simon/Seagate Expansion Drive/images_spanner' #local
    
    image_dataset = CustomImageDataset(attribute, img_dir, transform=data_transform)
    dataloader = DataLoader(image_dataset, batch_size=batch_size, shuffle=False)

    dataset_size = len(image_dataset)

    return dataloader, dataset_size

In [ ]:
def make(config, model_name):

    # JUST LOAD THE .pth FILE but you will still need the wieghts from the original model for the base transformations

    # Make the model
    weight_dict, model_dict = get_models()

    #Choose model and wieghts
    weights = weight_dict[model_name] # you need these later right as they hold the appropiate data transforamtions
    model = model_dict[model_name].to(device)
    # wandb.watch(model)

    # NO!
    # new model head for for retraining
    change_head(model_name, model, config['classes'])

    # NO!
    # re-train all parameters
    for param in list(model.parameters()):
        param.requires_grad = True
    
    # Make the data
    dataloaders, dataset_sizes = make_loader(batch_size=config['batch_size'],  weights = weights, attribute = config['attribute'])
    
    # NO NEED!
    # Make the loss and optimizer
    #criterion = nn.CrossEntropyLoss()
    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=config['learning_rate'], weight_decay = config['weight_decay'], betas = config['betas'])

    return model, criterion, optimizer, dataloaders, dataset_sizes #, class_names

In [ ]:
# IDK

def train_log(loss, example_ct, epoch):
    # Where the magic happens
    # wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")


In [ ]:
# NO NEED!!!!

def train_batch(images, labels, model, optimizer, criterion):
    images, labels = images.to(device), labels.to(device)
    
    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs.squeeze(), labels)
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss


In [ ]:
# NO NEED!!!

def train(model, loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    #wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config['epochs']
    example_ct = 0  # number of examples seen
    batch_ct = 0
    running_loss = 0.0

    for epoch in range(config['epochs']):
        for _, (images, labels) in enumerate(loader):

            loss = train_batch(images, labels, model, optimizer, criterion)
            example_ct +=  len(images)
            batch_ct += 1

            running_loss += loss

            # Report metrics every 20th batch - not running average right now
            if ((batch_ct + 1) % 100) == 0:
                #train_log(loss, example_ct, epoch) # this is the wand part
                train_log(running_loss/100, example_ct, epoch)
                running_loss = 0.0 # reset


In [ ]:
# THIS IS THE CORE NOW! Save each out put with the image id. - until now you have not used the image id..

def test(model, test_loader):
    model.eval()
    test_criterion = nn.MSELoss()

    # Run the model on some test examples
    with torch.no_grad():
        #correct, total = 0, 0
        total = 0
        RMSE_loss = 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            #_, predicted = torch.max(outputs.data, 1)
            RMSE_loss += torch.sqrt(test_criterion(outputs.squeeze().cpu(), labels.cpu()))

            total += 1 #labels.size(0)
            #correct += (predicted == labels).sum().item()

        # print(f"Accuracy of the model on the {total} " +
        #       f"test images: {100 * correct / total}%")
        
        print(f"Average RMSE of the model on the {total} " +
              f"test images: {RMSE_loss / total}%")


        #wandb.log({"test_rmse": RMSE_loss / total})

    # Save the model in the exchangeable ONNX format
    #torch.onnx.export(model, images, "model.onnx")
    #wandb.save("model.onnx")

In [ ]:
# access all HPs through wandb.config, so logging matches execution!
config = hyperparameters

model_name = config['model_name']

# make the model, data, and optimization problem
model, criterion, optimizer, dataloaders, dataset_sizes = make(config, model_name)
print(model)

In [ ]:
# # and use them to train the model
# train(model, dataloaders['train'], criterion, optimizer, config)

# # and test its final performance
# test(model, dataloaders['test'])